# Traffic Death Analysis

### Exercise 1: downloading and exploring the data.

In [1]:
import pandas as pd
import numpy as np
df=pd.read_excel("taxes.xlsx")

In [2]:
df.head(5)

,state,year,spirits,unemp,income,emppop,beertax,baptist,mormon,drinkage,...,nfatal2124,afatal,pop,pop1517,pop1820,pop2124,milestot,unempus,emppopus,gsp
0,al,1982,1.37,14.4,10544.15234,50.692039,1.539379,30.355700,0.32829,19.00,...,32,309.437988,3942002.25,208999.5938,221553.4375,290000.0625,28516.0,9.7,57.799999,-0.022125
1,al,1983,1.36,13.7,10732.79785,52.147030,1.788991,30.333599,0.34341,19.00,...,35,341.834015,3960008.00,202000.0781,219125.4688,290000.1563,31032.0,9.6,57.900002,0.046558
2,al,1984,1.32,11.1,11108.79102,54.168087,1.714286,30.311501,0.35924,19.00,...,34,304.872009,3988991.75,196999.9688,216724.0938,288000.1563,32961.0,7.5,59.500004,0.062798
3,al,1985,1.28,8.9,11332.62695,55.271137,1.652542,30.289499,0.37579,19.67,...,45,276.742004,4021007.75,194999.7344,214349.0313,284000.3125,35091.0,7.2,60.100002,0.027490
4,al,1986,1.23,9.8,11661.50684,56.514496,1.609907,30.267401,0.39311,21.00,...,29,360.716003,4049993.75,203999.8906,212000.0000,263000.2813,36259.0,7.0,60.700001,0.032143


In [3]:
states=len(np.unique(df['state']))
max_year=np.max(df['year'])
min_year=np.min(df['year'])
time=max_year-min_year
print (f'The data set contains {states} unique values. It was collected over {time} years of observations from {min_year} to {max_year}. Each unit of observations represents an aggregated data per state per year.')


The data set contains 48 unique values. It was collected over 6 years of observations from 1982 to 1988. Each unit of observations represents an aggregated data per state per year.


### Exercise 2: constructing the dependent variable (fatality rate per 10 000 people).

In [4]:
#Renaming pop column to population.
df.rename(columns={'pop':'population'}, inplace=True)
#Calculate fatality rate.
df['fat_rate']=df['fatal']/df['population']*10000

### Exercise 3: plotting beer tax value vs fatality rate.

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
ax=sns.regplot(x="beertax", y="fat_rate", data=df, line_kws={"color":"g","alpha":0.7,"lw":5})
ax.set_xlabel('Beer tax',fontsize=20);
ax.set_ylabel('Fatality rate',fontsize=20);
ax.set_title('Fatality rate vs beer taxes for 1982-1986',fontsize=20);
plt.show()

<Figure size 1000x800 with 1 Axes>

### Exercise 4: regres fatality rate on beer taxes.

In [7]:
import statsmodels.formula.api as smf
model = smf.ols('fat_rate ~ beertax', df).fit() 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               fat_rate   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     34.39
Date:                Tue, 25 Feb 2020   Prob (F-statistic):           1.08e-08
Time:                        20:24:47   Log-Likelihood:                -271.04
No. Observations:                 336   AIC:                             546.1
Df Residuals:                     334   BIC:                             553.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8533      0.044     42.539      0.000       1.768       1.939
beertax        0.3646      0.062      5.865      0.000       0.242       0.487
==============================================================================
Omnibus:                       66.653   Durbin-Watson:                   0.465
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              112.734
Skew:                           1.134   Prob(JB):                     3.31e-25
Kurtosis:                       4.707   Cond. No.                         2.76
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The average number of death per year is 1.8533 and with each incremental increase in beer taxes, the number of death increases by 0.3646 deaths. In other words, an increase in taxes leads to a higher number of deaths.

### Exercise 5: regress fatality rate on beer taxes and add state fixed effects.

In [8]:
model = smf.ols('fat_rate ~ beertax+C(state)', df).fit() 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               fat_rate   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                     56.97
Date:                Tue, 25 Feb 2020   Prob (F-statistic):          1.96e-120
Time:                        20:24:54   Log-Likelihood:                 107.97
No. Observations:                 336   AIC:                            -117.9
Df Residuals:                     287   BIC:                             69.09
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          3.4776      0.313     11.098      0.000       2.861       4.094
C(state)[T.ar]    -0.6550      0.219     -2.990      0.003      -1.086      -0.224
C(state)[T.az]    -0.5677      0.267     -2.129      0.034      -1.093      -0.043
C(state)[T.ca]    -1.5095      0.304     -4.960      0.000      -2.109      -0.910
C(state)[T.co]    -1.4843      0.287     -5.165      0.000      -2.050      -0.919
C(state)[T.ct]    -1.8623      0.281     -6.638      0.000      -2.414      -1.310
C(state)[T.de]    -1.3076      0.294     -4.448      0.000      -1.886      -0.729
C(state)[T.fl]    -0.2681      0.139     -1.924      0.055      -0.542       0.006
C(state)[T.ga]     0.5246      0.184      2.852      0.005       0.163       0.887
C(state)[T.ia]    -1.5439      0.253     -6.092      0.000      -2.043      -1.045
C(state)[T.id]    -0.6690      0.258     -2.593      0.010      -1.177      -0.161
C(state)[T.il]    -1.9616      0.291     -6.730      0.000      -2.535      -1.388
C(state)[T.in]    -1.4615      0.273     -5.363      0.000      -1.998      -0.925
C(state)[T.ks]    -1.2232      0.245     -4.984      0.000      -1.706      -0.740
C(state)[T.ky]    -1.2175      0.287     -4.240      0.000      -1.783      -0.652
C(state)[T.la]    -0.8471      0.189     -4.490      0.000      -1.218      -0.476
C(state)[T.ma]    -2.1097      0.276     -7.641      0.000      -2.653      -1.566
C(state)[T.md]    -1.7064      0.283     -6.025      0.000      -2.264      -1.149
C(state)[T.me]    -1.1079      0.191     -5.797      0.000      -1.484      -0.732
C(state)[T.mi]    -1.4845      0.236     -6.290      0.000      -1.949      -1.020
C(state)[T.mn]    -1.8972      0.265     -7.157      0.000      -2.419      -1.375
C(state)[T.mo]    -1.2963      0.267     -4.861      0.000      -1.821      -0.771
C(state)[T.ms]    -0.0291      0.148     -0.196      0.845      -0.321       0.263
C(state)[T.mt]    -0.3604      0.264     -1.365      0.173      -0.880       0.159
C(state)[T.nc]    -0.2905      0.120     -2.424      0.016      -0.526      -0.055
C(state)[T.nd]    -1.6234      0.254     -6.396      0.000      -2.123      -1.124
C(state)[T.ne]    -1.5222      0.249     -6.106      0.000      -2.013      -1.032
C(state)[T.nh]    -1.2545      0.210     -5.983      0.000      -1.667      -0.842
C(state)[T.nj]    -2.1057      0.307     -6.855      0.000      -2.710      -1.501
C(state)[T.nm]     0.4264      0.254      1.677      0.095      -0.074       0.927
C(state)[T.nv]    -0.6008      0.286     -2.101      0.037      -1.164      -0.038
C(state)[T.ny]    -2.1867      0.299     -7.316      0.000      -2.775      -1.598
C(state)[T.oh]    -1.6744      0.254     -6.597      0.000      -2.174      -1.175
C(state)[T.ok]    -0.5451      0.169     -3.223      0.001      -0.878      -0.212
C(state)[T.o

Adding fixed effects allows us to explore the relationship between beer taxes and the number of death within each state. This time, we observe a negative coefficient for the beer tax, imlying that the number of death actually went down.

### Exercise 6

Potentially, we miscalculated the effect for the first time, because the policy was not effective in the Southern States (Georgia, New Mexico, South Carolina).

### Exercise 6.2: implement the entity-demeaned approach.

In [9]:
from pandasql import sqldf
avg_df=sqldf("SELECT state, avg(beertax) as avgtax, fat_rate, avg(fat_rate) as avg_fat_rate from df GROUP BY state")
total=df.merge(avg_df,on='state', how='left')
total['demean_fat_rate']=total['fat_rate_x']-total['avg_fat_rate']
total['demean_beertax']=total['beertax']-total['avgtax']

In [10]:
model = smf.ols('demean_fat_rate ~ demean_beertax', total).fit() 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        demean_fat_rate   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     14.19
Date:                Tue, 25 Feb 2020   Prob (F-statistic):           0.000196
Time:                        20:24:58   Log-Likelihood:                 107.97
No. Observations:                 336   AIC:                            -211.9
Df Residuals:                     334   BIC:                            -204.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -1.301e-17      0.010  -1.36e-15      1.000      -0.019       0.019
demean_beertax    -0.6559      0.174     -3.767      0.000      -0.998      -0.313
==============================================================================
Omnibus:                       53.045   Durbin-Watson:                   1.517
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              219.863
Skew:                           0.585   Prob(JB):                     1.81e-48
Kurtosis:                       6.786   Cond. No.                         18.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

A negative coefficient for the demeaned beer tax variable suggests that the policy was indeed effectie. The mortality rate decreases, approximately by 0.66 deaths, with each unit of increase for beer taxes.

### Exercise 7: fit the model with state fixed-effect using PanelOLS / lfe.

In [14]:
from linearmodels import PanelOLS
stemp = df.set_index(['state', 'year'])
model = PanelOLS.from_formula('fat_rate ~ beertax + EntityEffects', stemp)
model.fit(cov_type='clustered', cluster_entity=True)

Dep. Variable:,fat_rate,R-squared:,0.0407
Estimator:,PanelOLS,R-squared (Between):,-0.3805
No. Observations:,336,R-squared (Within):,0.0407
Date:,"Tue, Feb 25 2020",R-squared (Overall):,-0.3775
Time:,20:26:28,Log-likelihood,107.97
Cov. Estimator:,Clustered,,
,,F-statistic:,12.190
Entities:,48,P-value,0.0006
Avg Obs:,7.0000,Distribution:,"F(1,287)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,5.1576


The resulting coefficient is exactly the same as we obtained using demeaning: an increase in taxes indeed leads to lower fatality rates, though standard error is lower.

### Exercise 8: estimate a fixed effects model.

In [13]:
from linearmodels import PanelOLS
temp = df.set_index(['state','year'])
model = PanelOLS.from_formula('fat_rate ~ beertax + youngdrivers+EntityEffects', temp)
model.fit(cov_type='clustered', cluster_entity=True)

Dep. Variable:,fat_rate,R-squared:,0.0493
Estimator:,PanelOLS,R-squared (Between):,-0.2476
No. Observations:,336,R-squared (Within):,0.0493
Date:,"Tue, Feb 25 2020",R-squared (Overall):,-0.2455
Time:,20:21:40,Log-likelihood,109.48
Cov. Estimator:,Clustered,,
,,F-statistic:,7.4207
Entities:,48,P-value,0.0007
Avg Obs:,7.0000,Distribution:,"F(2,286)"
Min Obs:,7.0000,,
Max Obs:,7.0000,F-statistic (robust):,2.4349


a). By adding the year fixed effect, we able to explore the relationship between beer taxes and the number of death within each state and within each year. b). We were not provided with a codebook, so we can be mistaken in variables interpretation. However, we assume that "youngdrivers" is the variable representing the proportion of young drivers among all drivers - population that is less experienced in driving and more likely to take a risk of driving under the influence.

An increase in beer taxes by one unit indeed decreases the fatality rate by 0.7374 death. Our guess about the young drivers was also correct - with each unit increase in your drivers' population, fatality rates increase by 0.9491 deaths.